# **전처리**
0. 라이브러리 호출
1. 초기 전처리
2. 키워드 정제 및 카테고리 분류
3. 중복 제거 및 고유 게임 파악
4. [게임 정보] 테이블
5. [지점별 보유 게임] 테이블
6. [지점 정보] 테이블
7. [지점별 보유 게임] 테이블에 외래키 컬럼 추가
8. 테이블 확인
9. 테이블 저장

<br>

# **0. 라이브러리 호출**

In [3]:
import pandas as pd
import numpy as np
import re

<br>

## **1. 초기 전처리**
- CSV 파일 로딩
- 결측치/이상치 처리
- 지점명 정제

In [5]:
# CSV 파일 로딩
df = pd.read_csv('redbutton_web_data_crawling_result.csv', encoding='cp949')

# 결측치/이상치 처리
df.loc[df['게임명'] == '핏츠', ['인원', '게임시간', '키워드']] = ['1-4인', '45분', '퍼즐']

# 지점명 정제
df['지점'] = np.where(
    df['지점'].str.endswith('점', na=False),
    df['지점'].str[:-1],
    df['지점']
)

df

,지역,지점,게임명,난이도,인원,게임시간,설명영상,키워드
0,서울,신촌,세븐원더스 : 듀얼,Extreme,2인,60분,아니오,셋컬렉션
1,서울,신촌,도블,Easy,2-5인,10분,예,순발력
2,서울,신촌,데어이쎄스,Easy,2인,5분,예,심리추리
3,서울,신촌,데드오브윈터,ProGamer,2-5인,100분,아니오,미션협력
4,서울,신촌,더게임,Normal,2-5인,20분,예,숫자협력
...,...,...,...,...,...,...,...,...
35563,제주,제주시청,캣츠런,Very Easy,2-4인,10분,예,"주사위, 경주"
35564,제주,제주시청,풍기,Hard,2인,30분,아니오,카드 전략
35565,제주,제주시청,나의 최애 명대사,Easy,3-6인,15분,아니오,"순발력, 명대사"
35566,제주,제주시청,라이헌트 미스터리 아티스트,Easy,3-8인,30분,예,그림/마피아


<br>

## **2. 키워드 정제 및 카테고리 분류**
- 키워드 전처리 함수 정의
- 오탈자 수정 및 표준화 함수 정의
- 대표 키워드 추출 및 상위 카테고리 생성
- 상위 카테고리 매핑

In [7]:
# 키워드 전처리 함수
# - 특수문자 제거 + 분리
def clean_and_split_keywords(keyword):
    if pd.isna(keyword):
        return []
    
    # 특수 문자 제거 및 구분자 통일
    cleaned = re.sub(r'[#/]', '', keyword)        # '#'와 '/' 제거
    cleaned = re.sub(r'[·|&]', ',', cleaned)      # '·' 또는 '&' 또는 '|' → ','
    cleaned = cleaned.replace(' ', '')            # 공백 제거
    split_keywords = re.split(r'[,\n]', cleaned)  # 쉼표 또는 줄바꿈으로 분리
    return [kw for kw in split_keywords if kw]

# 오탈자 수정 함수
def fix_keyword(keyword):
    if pd.isna(keyword):
        return ''

    # 완전 일치 시에만 바꾸는 딕셔너리(단순 오기)
    exact_match_map = {
        '엔진빌': '엔진빌딩',
        '이야기만들': '이야기',
        '순발': '순발력',
        '리얼타임순발력': '순발력',
        '복물복': '복불복',
        '힘대결': '카드',
        '미정': '퍼즐'   # 핏츠 키워드기 '미정'인데, 핏츠는 퍼즐 게임
    }

    # 포함되면 바꾸는 딕셔너리(용어 미통일)
    partial_match_map = {
        '컬랙션': '컬렉션',
        '콜랙션': '컬렉션',
        '배팅': '베팅',
        '미니어쳐': '미니어처',
        '셋': '세트',
        '머더미스터리': '머더'
    }

    # --- 1. 완전 일치 처리 ---
    if keyword in exact_match_map:
        return exact_match_map[keyword]

    # --- 2. 부분 포함 처리 ---
    for wrong, correct in partial_match_map.items():
        if wrong in keyword:
            keyword = keyword.replace(wrong, correct)

    return keyword
    
# 분리된 키워드 컬럼 추가
df['분리된_키워드'] = df['키워드'].apply(clean_and_split_keywords)

# 대표 키워드 (리스트의 첫 번째 항목)
df['대표_키워드'] = df['분리된_키워드'].apply(lambda x: x[0] if x else '')

# 적용
df['카테고리'] = df['대표_키워드'].apply(fix_keyword)

# 불필요한 컬럼 제거
df = df.drop(['키워드', '분리된_키워드', '대표_키워드'], axis=1)

df

,지역,지점,게임명,난이도,인원,게임시간,설명영상,카테고리
0,서울,신촌,세븐원더스 : 듀얼,Extreme,2인,60분,아니오,세트컬렉션
1,서울,신촌,도블,Easy,2-5인,10분,예,순발력
2,서울,신촌,데어이쎄스,Easy,2인,5분,예,심리추리
3,서울,신촌,데드오브윈터,ProGamer,2-5인,100분,아니오,미션협력
4,서울,신촌,더게임,Normal,2-5인,20분,예,숫자협력
...,...,...,...,...,...,...,...,...
35563,제주,제주시청,캣츠런,Very Easy,2-4인,10분,예,주사위
35564,제주,제주시청,풍기,Hard,2인,30분,아니오,카드전략
35565,제주,제주시청,나의 최애 명대사,Easy,3-6인,15분,아니오,순발력
35566,제주,제주시청,라이헌트 미스터리 아티스트,Easy,3-8인,30분,예,그림마피아


<br>

- 대표 키워드 추출은 `category_preprocessing.ipynb` 파일에서 진행
- 진행 결과를 수기로 추가 작업하여 15개 상위 카테고리로 생성 후 매핑 csv 파일 생성

In [9]:
# 대표 키워드 추출 및 상위 카테고리 생성
mapping_df = pd.read_csv('category_mapping_table.csv', encoding='cp949')

# 1. 엑셀에서 매핑 딕셔너리 생성
mapping_dict = dict(zip(mapping_df['하위카테고리'], mapping_df['상위카테고리']))

# 2. 기존 df에 매핑 적용
df['상위카테고리'] = df['카테고리'].map(mapping_dict)

df

,지역,지점,게임명,난이도,인원,게임시간,설명영상,카테고리,상위카테고리
0,서울,신촌,세븐원더스 : 듀얼,Extreme,2인,60분,아니오,세트컬렉션,전략-전쟁-경영
1,서울,신촌,도블,Easy,2-5인,10분,예,순발력,순발력-손기술
2,서울,신촌,데어이쎄스,Easy,2인,5분,예,심리추리,추리-마피아-심리
3,서울,신촌,데드오브윈터,ProGamer,2-5인,100분,아니오,미션협력,협력-팀플레이
4,서울,신촌,더게임,Normal,2-5인,20분,예,숫자협력,단어-퀴즈-숫자-지식
...,...,...,...,...,...,...,...,...,...
35563,제주,제주시청,캣츠런,Very Easy,2-4인,10분,예,주사위,주사위
35564,제주,제주시청,풍기,Hard,2인,30분,아니오,카드전략,카드-덱빌딩
35565,제주,제주시청,나의 최애 명대사,Easy,3-6인,15분,아니오,순발력,순발력-손기술
35566,제주,제주시청,라이헌트 미스터리 아티스트,Easy,3-8인,30분,예,그림마피아,그림


<br>

## **3. 중복 제거 및 고유 게임 파악**
- 지역-지점-게임명 기준 중복 제거

In [11]:
# 중복 행 제거
# 중복 판단 기준 컬럼들
cols = ['지역', '지점', '게임명']

# 중복 제거: 동일한 값이 있는 경우 첫 번째만 남기고 제거
df = df.drop_duplicates(subset=cols, keep='first').reset_index(drop=True)

df

,지역,지점,게임명,난이도,인원,게임시간,설명영상,카테고리,상위카테고리
0,서울,신촌,세븐원더스 : 듀얼,Extreme,2인,60분,아니오,세트컬렉션,전략-전쟁-경영
1,서울,신촌,도블,Easy,2-5인,10분,예,순발력,순발력-손기술
2,서울,신촌,데어이쎄스,Easy,2인,5분,예,심리추리,추리-마피아-심리
3,서울,신촌,데드오브윈터,ProGamer,2-5인,100분,아니오,미션협력,협력-팀플레이
4,서울,신촌,더게임,Normal,2-5인,20분,예,숫자협력,단어-퀴즈-숫자-지식
...,...,...,...,...,...,...,...,...,...
35410,제주,제주시청,캣츠런,Very Easy,2-4인,10분,예,주사위,주사위
35411,제주,제주시청,풍기,Hard,2인,30분,아니오,카드전략,카드-덱빌딩
35412,제주,제주시청,나의 최애 명대사,Easy,3-6인,15분,아니오,순발력,순발력-손기술
35413,제주,제주시청,라이헌트 미스터리 아티스트,Easy,3-8인,30분,예,그림마피아,그림


<br>

## **4. [게임 정보] 테이블**
- 중복 제거 및 게임 단위 정리
- 라벨 인코딩(난이도)
- 게임 시간 정제
- 최소/최대인원 추출
- 섦여영상 보우 여부 처리
- 이달의 게임 여부 추가
- 컬럼명 영문화

### **컬럼**
- 인덱스(gameId)
- 게임명(gameName)
- 난이도(gameLevel)
- 게임시간(gameTime)
- 설명영상(gameRule)
- 카테고리(gameCategory)
- 상위카테고리(parentCategory)
- 난이도_라벨링(gameLevel_Encoding)
- 최소인원(gamePlayerMin)
- 최대인원(gamePlayerMax)
- 이달의게임여부(monthlyGame)

In [14]:
# 최소/최대 인원 추출 함수 정의
def extract_min_max(person_str):
    if pd.isna(person_str):
        return pd.Series([pd.NA, pd.NA], dtype='Int64')

    cleaned = person_str.replace('명', '').replace('인', '').replace(' ', '')
    cleaned = cleaned.replace('~', '-').replace(',', '-')
    numbers = re.findall(r'\d+', cleaned)
    numbers = [int(n) for n in numbers]

    if len(numbers) == 0:
        return pd.Series([pd.NA, pd.NA], dtype='Int64')
    elif len(numbers) == 1:
        return pd.Series([numbers[0], numbers[0]], dtype='Int64')
    else:
        return pd.Series([min(numbers), max(numbers)], dtype='Int64')

gameinfo = df.drop_duplicates(subset=['게임명']).drop(['지역','지점'], axis=1).copy()

# 난이도 순서 정의
difficulty_order = ['Very Easy', 'Easy', 'Normal', 'Hard', 'Very Hard', 'Extreme', 'ProGamer']

# 난이도 컬럼을 카테고리형으로 변환하며 순서 지정
gameinfo['난이도'] = pd.Categorical(gameinfo['난이도'], categories=difficulty_order, ordered=True)

# 라벨 인코딩 (카테고리 코드로 변환)
gameinfo['난이도_라벨링'] = gameinfo['난이도'].cat.codes

# 게임시간(단위제거, 수치변환)
gameinfo['게임시간'] = gameinfo['게임시간'].str.replace('분', '', regex=False).astype(int)

# 최소/최대 인원
gameinfo[['최소인원', '최대인원']] = gameinfo['인원'].apply(extract_min_max)
gameinfo = gameinfo.drop(['인원'], axis=1)

# 설명영상
gameinfo['설명영상'] = gameinfo['설명영상'].map({'예': 1, '아니오': 0}).astype(bool)

# 이달의 게임 목록
monthly_games = [
    "라이헌트", "다빈치코드", "금지어 게임", "루미큐브",
    "라쿠카라차", "클루", "스플렌더", "선물입니다",
    "인디언포커", "꼬치의 달인"
]

# 이달의게임여부 조건 분기 처리
gameinfo['이달의게임여부'] = gameinfo['게임명'].apply(
    lambda x: x.strip().startswith("라이헌트") if x.strip().startswith("라이헌트")
    else x.strip() in monthly_games
)

# 게임명 기준 오름차순으로 정렬 후 gameId 부여
gameinfo.sort_values(by='게임명', inplace=True)
gameinfo['gameId'] = gameinfo['게임명'].astype('category').cat.codes
gameinfo = gameinfo[['gameId'] + [col for col in gameinfo.columns if col != 'gameId']]
gameinfo.reset_index(drop=True, inplace=True)

# 컬럼명 교체
gameinfo = gameinfo.rename(columns={
    '게임명': 'gameName',
    '난이도': 'gameLevel',
    '게임시간': 'gameTime',
    '설명영상' : 'gameRule',
    '카테고리': 'gameCategory',
    '상위카테고리': 'parentCategory',
    '난이도_라벨링': 'gameLevel_Encoding',
    '최소인원': 'gamePlayerMin',
    '최대인원': 'gamePlayerMax',
    '이달의게임여부': 'monthlyGame',
})

gameinfo 

,gameId,gameName,gameLevel,gameTime,gameRule,gameCategory,parentCategory,gameLevel_Encoding,gamePlayerMin,gamePlayerMax,monthlyGame
0,0,1%의 기적,Very Easy,20,False,운빨,파티-가족-미니게임,0,2,6,False
1,1,10일간의 미국여행,Normal,30,False,세트컬렉션,전략-전쟁-경영,2,2,4,False
2,2,10일간의 유럽여행,Normal,30,False,세트컬렉션,전략-전쟁-경영,2,2,4,False
3,3,13클루,Normal,30,True,카드추리,카드-덱빌딩,2,3,6,False
4,4,3초 트라이,Easy,15,False,퀴즈,단어-퀴즈-숫자-지식,1,3,6,False
...,...,...,...,...,...,...,...,...,...,...,...
1061,1061,흔들흔들 밸런스타워,Very Easy,15,False,균형잡기,공간전략,0,2,4,False
1062,1062,흔들흔들해적선,Very Easy,5,True,손기술,순발력-손기술,0,2,4,False
1063,1063,히트 질주의 열기,Very Hard,45,False,레이싱,경주-스포츠,4,1,6,False
1064,1064,히트 폭우를 뚫고,Hard,60,False,레이싱,경주-스포츠,3,2,7,False


<br>

## **5. [보유 게임] 테이블**
- 불필요 컬럼 삭제
- 컬럼명 영문화


### **컬럼**
- 지역(area)
- 지점명(location)
- 게임 이름(gameName)

In [16]:
# 보유 게임 목록 정리
taken = df.copy()
taken = taken.drop(['지역', '난이도','인원','게임시간', '설명영상', '카테고리', '상위카테고리'], axis=1)
taken = taken.rename(columns={'지점': 'location', 
                              '게임명': 'gameName'})
taken["gameName"] = taken["gameName"].str.strip()

# 이달의 게임 보유 수 계산
monthly_game_names = gameinfo.query("monthlyGame == True")["gameName"].str.strip().tolist()
taken_monthly = taken[taken["gameName"].isin(monthly_game_names)]
monthly_counts = taken_monthly["location"].value_counts().reset_index()
monthly_counts.columns = ["location", "이달의게임수"]
monthly_counts.rename(columns={"location": "지점"}, inplace=True)

taken

,location,gameName
0,신촌,세븐원더스 : 듀얼
1,신촌,도블
2,신촌,데어이쎄스
3,신촌,데드오브윈터
4,신촌,더게임
...,...,...
35410,제주시청,캣츠런
35411,제주시청,풍기
35412,제주시청,나의 최애 명대사
35413,제주시청,라이헌트 미스터리 아티스트


- `taken` 테이블에는 `gameinfo` 테이블과 `stores` 테이블의 기본키를 외래키로 참조해함
- [지점 정보] 테이블 선언 후, [taken] 테이블에 컬럼 추가

<br>

## **6 [지점 정보] 테이블**
- 지점별 보유 게임 리스트 생성
- 보유/미보유 게임 수 계산
- 컬럼명 영문화

### **컬럼**
- 지역(area)
- 지점명(location)
- 보유 게임 개수(numOfTaken)
- 미보유 게임 개수(numOfUntaken)
- 이달의 게임 보유수(numOfMonthlyGames)

In [19]:
# 지점별 통계 정리
stores = df.copy()
stores = stores.drop_duplicates()
stores = stores.drop(['난이도','인원','게임시간','설명영상','카테고리'], axis = 1)
stores = stores.groupby(['지역', '지점'], sort=False)['게임명'].apply(list).reset_index()
stores['보유게임수'] = stores['게임명'].apply(len)

# 미보유 게임 계산
cols = ['게임명', '난이도', '인원', '게임시간', '설명영상', '카테고리']
unique_games = df[cols].drop_duplicates()
unique_count = unique_games.shape[0]
stores['미보유게임수'] = unique_count - stores['보유게임수']

# 이달의 게임 수 병합
stores = stores.drop(columns=[col for col in stores.columns if "이달의게임수" in col], errors="ignore")
stores = stores.merge(monthly_counts, on="지점", how="left")
stores["이달의게임수"] = stores["이달의게임수"].fillna(0).astype(int)
stores = stores.drop(['게임명'], axis = 1)

# 컬럼명 변경
stores = stores.rename(columns={
    '지역': 'area',
    '지점': 'location',
    '보유게임수': 'numOfTaken',
    '미보유게임수' : 'numOfUntaken',
    '이달의게임수' : 'numOfMonthlyGames'
})

# 기준 키 생성 (area + location)
store_keys = stores['area'] + '_' + stores['location']

# 순서를 유지하면서 고유 ID 부여 (딕셔너리 매핑 방식 사용)
unique_keys = pd.Series(store_keys.unique())
store_key_to_id = {key: i for i, key in enumerate(unique_keys)}
stores['store_id'] = store_keys.map(store_key_to_id)

stores

,area,location,numOfTaken,numOfUntaken,numOfMonthlyGames,store_id
0,서울,신촌,412,665,11,0
1,서울,신림,365,712,11,1
2,서울,대학로,424,653,11,2
3,서울,강남,406,671,11,3
4,서울,노원,346,731,11,4
...,...,...,...,...,...,...
99,부산,부산대,460,617,11,99
100,부산,부산사상,332,745,11,100
101,부산,광안리,300,777,11,101
102,부산,부산동래,322,755,11,102


## **7. [지점별 보유 게임] 테이블에 외래키 컬럼 추가**

In [21]:
# 1. gameName → gameId 매핑 테이블
gameinfo_fk_map = gameinfo[['gameName']].drop_duplicates().reset_index(drop=True)
gameinfo_fk_map['gameId'] = gameinfo_fk_map.index

# 2. location → storeId 매핑 테이블
stores_fk_map = stores[['location']].drop_duplicates().reset_index(drop=True)
stores_fk_map['storeId'] = stores_fk_map.index

# 3. taken 데이터프레임에 외래키 컬럼 직접 추가
taken = taken.merge(gameinfo_fk_map, on='gameName', how='left')
taken = taken.merge(stores_fk_map, on='location', how='left')

# 결과 확인
taken.head()

,location,gameName,gameId,storeId
0,신촌,세븐원더스 : 듀얼,448,0
1,신촌,도블,139,0
2,신촌,데어이쎄스,128,0
3,신촌,데드오브윈터,127,0
4,신촌,더게임,116,0


<br>

## **8. 테이블 확인**
- gameinfo
- stores
- taken

In [23]:
# gameinfo 테이블 확인
gameinfo.head()

,gameId,gameName,gameLevel,gameTime,gameRule,gameCategory,parentCategory,gameLevel_Encoding,gamePlayerMin,gamePlayerMax,monthlyGame
0,0,1%의 기적,Very Easy,20,False,운빨,파티-가족-미니게임,0,2,6,False
1,1,10일간의 미국여행,Normal,30,False,세트컬렉션,전략-전쟁-경영,2,2,4,False
2,2,10일간의 유럽여행,Normal,30,False,세트컬렉션,전략-전쟁-경영,2,2,4,False
3,3,13클루,Normal,30,True,카드추리,카드-덱빌딩,2,3,6,False
4,4,3초 트라이,Easy,15,False,퀴즈,단어-퀴즈-숫자-지식,1,3,6,False


In [24]:
# stores 테이블 확인
stores.head()
# print(stores.shape)

,area,location,numOfTaken,numOfUntaken,numOfMonthlyGames,store_id
0,서울,신촌,412,665,11,0
1,서울,신림,365,712,11,1
2,서울,대학로,424,653,11,2
3,서울,강남,406,671,11,3
4,서울,노원,346,731,11,4


In [25]:
# taken 테이블 확인
taken.head()

,location,gameName,gameId,storeId
0,신촌,세븐원더스 : 듀얼,448,0
1,신촌,도블,139,0
2,신촌,데어이쎄스,128,0
3,신촌,데드오브윈터,127,0
4,신촌,더게임,116,0


<br>

## **9. 테이블 저장**

In [27]:
gameinfo.to_csv("gameinfo_table.csv", index=False, encoding='cp949')

In [28]:
stores.to_csv('stores_table.csv', index=False, encoding='cp949')

In [29]:
taken.to_csv('taken_table.csv', index=False, encoding='cp949')